## IDF 04 Exercise

In this exercise we will implement a basic sEMG Analyis for 8-channel sEMG. You may use a data science framework of your choosing. The solution code will be provided in Python.

In StudOn you will find 2 text files you will need in this exercise. The `raw_EMG.txt` holds sEMG data from the Myo for the movements flexion, extension and rest. The `capture_data.txt` gives out the timestamps the recording.
"0" = "Rest", "1" = "Flexion", "2" = Extension. One movement recording always goes from start timestamp until the next "N/A" timestamp. e.g. the first "rest" goes from 21493.069105 until 21497.229725. 

If not otherwise specified, Numpy, Pandas, Matplotlib.Pyplot and Scipy.Signal (only Signal!) may be used (in case of python). 

### E4.1 - Load Data and Timer Sanity Check
Load the dataset of raw EMG values into `df_myo` and name the 9 columns to `['timestamp', 'EMG1','EMG2','EMG3','EMG4','EMG5','EMG6','EMG7','EMG8']`. Perform a "timer sanity check". To do so: check the difference between single timestamps. Calculate each `delta_t` between samples. Then calculate the mean `delta_t_mean`, and std `delta_t_std` of the dataset. Plot the single `delta_t`s with a horizontal reference line with respect to the sampling rate of $f_s = 200 Hz$. Add a reference line for the standard deviations (negative and positve of mean).

##### `code`

In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

from scipy import signal
# from sklearn.decomposition import PCA

from mlxtend.preprocessing import standardize

# EMG file
path = 'raw_EMG.txt'

ModuleNotFoundError: No module named 'mlxtend'

In [ ]:
#OPTIONAL

import ipympl
%matplotlib inline 


In [ ]:
df_myo = pd.read_csv(path, sep=",", header=None)
df_myo.columns = ['timestamp', 'EMG1','EMG2','EMG3','EMG4','EMG5','EMG6','EMG7','EMG8']
timestamps = df_myo['timestamp'].copy()
delta_t = timestamps.diff(periods=1)
delta_t_mean = delta_t.mean()
delta_t_std = delta_t.std()


In [ ]:
x_values = np.arange(0, len(delta_t)) # in samples

f1 = plt.figure()

plt.scatter(x_values, delta_t, s=5)
plt.axhline(y=delta_t_mean, color='r', linestyle='-')
plt.axhline(y=delta_t_mean+delta_t_std, color='y', linestyle='-')
plt.axhline(y=delta_t_mean-delta_t_std, color='y', linestyle='-')

plt.show()
print("mean Delta: ", delta_t_mean*10**6, " µs")
print("mean std: ", delta_t_std*10**6," µs")

### E4.2 - Rectified and Smoothed sEMG 

Each channel is a column of the dataset. Start by plotting the EMG values. For a better differentiation of the channels add a vertical offset of $lastChannel + 2$ to the channels in the plot. This will be our standard way of plotting sEMG in this exercise.

Similar to this:
![alt text](EMG2.png "Title")

In a next step rectify the complete signal and plot it again.

Afterwards run it through a butterworth low-pass filter with the following properties: 
sampling frequency $f_s = 200 Hz$ filter order $N = 2$; cuttoff frequency $f_c = 1 Hz $.
If you implement this in Python, please have a look at the documentation from scipy.signal for this. Especially have a look at the calculation of `Wn`. Get the parameters ``b, a`` and use the function `lfilter`. Don't worry, we will have a closer look at filtering in another exercise. 
Plot the result again.

##### `code`

In [ ]:
# plotting EMG lines
fig_raw_emg = plt.figure()
for i in range(1,len(df_myo.columns)):
    plt.plot(df_myo.iloc[:, i] + 2*i, linewidth=0.5)

data_df = df_myo.iloc[1:, 1:]

In [ ]:
# Rectification (taking the absolute value) without timestamps
df_myo_rec = df_myo.iloc[:, 1:].copy()
df_myo_rec = df_myo_rec.abs()

for i in range(1,len(df_myo_rec.columns)):
    plt.plot(df_myo_rec.iloc[:, i] + 2*i, linewidth=0.5)

In [ ]:
# Lowpass filter
f_s = 200 #Hz
f_c = 1 #Hz
N = 2 # filter order

# 1. calculate normed coefficients
Wn = 2 * f_c / f_s
b, a = signal.butter(N=2, Wn=Wn, btype="low")

# 2. apply filter
myo_lp = signal.lfilter(b,a, df_myo_rec.to_numpy(), axis=0)

for i in range(1,myo_lp.shape[1]):
    plt.plot(myo_lp[:, i] + 2*i, linewidth=0.5)



### E4.3 Splits

Load the `capture.txt`. Keep in mind that "0" = "Rest", "1" = "Flexion", "2" = Extension and the explanations of the splits above. Delete any non-labeled data in your EMG-dataframe with the information from the capture.txt. Define a new dataset ``df_movs``, which only includes labelled samples.

#### ``Code``

In [ ]:
splits_basic = pd.read_csv('capture_data.txt', ",", header=None, dtype=str)
splits_basic.columns = ['timestamp', 'movs']
splits_basic= splits_basic.astype(float)

# add timestamp back to lowpass filtered data
df_emg_tmp = pd.DataFrame(myo_lp)
df_timestamps =  pd.DataFrame(timestamps)

# lp with timestamp
df_emg_new = df_timestamps.join(df_emg_tmp)
df_emg_new.columns = ['timestamp', 'EMG1','EMG2','EMG3','EMG4','EMG5','EMG6','EMG7','EMG8']



##### `code`

In [ ]:
# new dataframe with only movs
df_movs = pd.DataFrame(columns = ['timestamp', 'EMG1','EMG2','EMG3','EMG4','EMG5','EMG6','EMG7','EMG8', 'movs'])

new_start = 0
label = ""

# concat movements
for index, row in splits_basic.iterrows():
    if(np.isnan(row["movs"])):
        #select rows
        new_end = row['timestamp']
        df_select = df_emg_new.loc[(df_emg_new['timestamp'] < new_end) & (df_emg_new['timestamp'] >= new_start)]
        df_select['movs'] = label
        
        # concat with existing df_movs
        df_movs = pd.concat([df_movs, df_select])

    else:
        new_start = row['timestamp']
        label = row['movs']

# reset index
df_movs.reset_index(inplace=True, drop=True)


### E4.4 (PCA) Standardization and Covariance Matrix
Standardize the dataset `df_movs` (center and standardize deviation) in the variable `df_movs_normed`. Please check the last exercise if you don't remember the single steps. Keep in mind that we standardize with $n-1$. 

A `standardize` function e.g. `mlxtend.preprocessing` may be used. Then calculate the covariance matrix `cov_mat` of the new dataset. A function to calculate the covariance matrix e.g. `numpy.cov` may be used. 

#### ``Code``

In [ ]:
# standardization
df_movs_normed = df_movs.copy()
df_movs_normed.iloc[:, 1:-1] = standardize(df_movs_normed.iloc[:, 1:-1], ddof=1)

# only values (no timestamp or mov)
movs_normed = np.array(df_movs_normed.iloc[:, 1:-1])

# covariance matrix
cov_mat = np.cov(df_movs_normed.iloc[:, 1:-1], rowvar=False, ddof=1)

### E4.5 (PCA) Eigenvalues/-vectors of covariance matrix
Calculate the Eigenvalues `eig_vals` and -vectors `eig_vecs` of the covariance matrix. A function to get the covariance e.g. `numpy.linalg.eig` may be used. Follow the steps from the last exercise. Order the vectors into a matrix depending on their Eigenvalue. Shorten the resulting transformation matrix $T$ to 2 columns ($8 \times 8$ ~> $ 8 \times 2 $)

#### ``Code``

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(cov_mat)

# sorting (in this case not necessary)
idx = eig_vals.argsort()[::-1]
eig_vals_sort = eig_vals[idx]
eig_vecs_sort = eig_vecs[:,idx]

# get T by taking the first 2 columns of eig_vec_sort
T = eig_vecs_sort[:, :2]

### 4.6 (PCA) Data Transfomation 

Transform the EMG-dataset `df_movs_normed` (movements only) and get `df_movs_trans` with the transformation matrix $T$ and plot it in a 2D scatter plot with different colors for each movment. Afterwards calculate the explained variance `explained_var` with the help of the Eigenvalues `eig_vals` of the covariance matrix. Check last week's exercise if you don't rememeber the details.


#### ``Code``

In [ ]:
# transformation
movs_trans = movs_normed.dot(T)

# add movements again
df_movs_trans = pd.DataFrame(movs_trans)
df_movs_only = pd.DataFrame(df_movs_normed['movs'])
df_trans = df_movs_only.join(df_movs_trans)



In [ ]:
# option 2
fig_pca2 = plt.figure()
rest_ids = np.where(df_trans["movs"] == 0)
flex_ids = np.where(df_trans["movs"] == 1)
ext_ids = np.where(df_trans["movs"] == 2)

x_ = np.array(movs_trans[:,0])
y_ = np.array(movs_trans[:,1])

plt.scatter(x_[rest_ids], y_[rest_ids], label="rest")
plt.scatter(x_[flex_ids], y_[flex_ids], label="flexion")
plt.scatter(x_[ext_ids], y_[ext_ids], label = "extension")

plt.legend()

In [ ]:
# explained variance
explained_var = np.sum(eig_vals[:2]) / np.sum(eig_vals)
print(explained_var)

In [ ]:
plt.close()

# CHECK ONLY:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
principalComponents = pca.fit_transform(df_movs_normed.iloc[:, 1:-1])
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

plt.scatter(principalComponents[:,0],principalComponents[:,1])
plt.show()

#check

var = np.sum(pca.explained_variance_ratio_)

### optional - 3D PCA

Implememnt a 3D PCA, plot it and calculate the explained variance. Then compare it last result above. 